The following code connect to the Data Mill North page, parse the html and process the csv files uploaded by LSC.  It is adapted from Nic Mallesons initial exploration of the data found at https://github.com/Urban-Analytics/dust/blob/main/Projects/Ambient_Populations/AmbientPopulations.ipynb.
There are various checks to ensure duplicate files are not downloaded and merged into the final dataframe.  Initially the code included a check on the filename to filter out anything that started with 'Copy of', however after visualising the data I discovered that a lot of the data was missing from
earlier years (mostly 2015-2017) as many of the files had been named 'Copy of....' yet were not duplicates.  The code already ensures files that exist are not downloaded and I've gone through and eyeballed the files to do a sense check of whether duplicates exist or not.

In [1]:
import plotly.express as px

import matplotlib.ticker as mticker
from bs4 import BeautifulSoup  # requirement beautifulsoup4
from urllib.request import (
    urlopen, urlretrieve)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import os, os.path
import sys
import pandas as pd
import numpy as np
from IPython.display import HTML

from source import *

data_dir = "./data/lcc_footfall"  # Where to save the csv files

#Function to parse the html and download the csv files to specified location
download_data(data_dir)
footfalldf_imported = import_data(data_dir)

File Copy%20of%20Monthly%20Data%20Feed%20-%20Oct%202017.csv has nans in the following columns: '['Hour']'. Ignoring it
File Copy%20of%20Monthly%20Data%20Feed%20-%20Sept%202017.csv has nans in the following columns: '['Hour']'. Ignoring it
File Copy%20of%20Monthly%20Data%20Feed-November%202016%20-%2020161221.csv has nans in the following columns: '['Date', 'InCount', 'Hour', 'Location']'. Ignoring it
File Monthly%20Data%20Feed%20-%20%20Jan%202018.csv has nans in the following columns: '['Hour']'. Ignoring it
File Monthly%20Data%20Feed%20-%20Dec%202017.csv has nans in the following columns: '['Hour']'. Ignoring it
File Monthly%20Data%20Feed%20-%20Nov%202017.csv has nans in the following columns: '['Hour']'. Ignoring it
File Monthly%20Data%20Feed-April%202017%20-%2020170510.csv has nans in the following columns: '['Date', 'InCount', 'Hour', 'Location']'. Ignoring it
File Monthly%20Data%20Feed-Feb%202018.csv has nans in the following columns: '['Hour']'. Ignoring it
File Monthly%20Data%20F

Let's import the csv data files and clean the resulting dataframe based on some initial exploration.

First, the import_data function runs through each csv, creates a dataframe for each and then merges them.  Columns are converted to appropriate data types and any mismatches are fixed before merging.  This is important as Leeds City Council changed the formats of the files several times, which led to some differences in column names and potentially data types.

The next step in the pipeline is to check for duplicates and remove them.  Initial data exploration revealed errors in some of the csv files where individual records had been duplicated.  In some instances, the same records existed in several different files, for example dates in early July appeared towards the end of the June csv.

The cameras don't all come online at the same time, with the last starting on 27th August 2008.  To ensure meaningful comparability, any records before this date have been removed.

Finally, one of the cameras appeared to have moved locations on 31st May 2015 from Commercial Street at Lush to Commercial Street at Sharps.  These are combined and renamed to Commercial Street Combined.

In [2]:
#Pipeline that imports csv files, creates a dataframe and applies cleaning functions
footfalldf = (footfalldf_imported
              .pipe(start_pipeline)
              .pipe(set_start_date)
              .pipe(combine_cameras)
              .pipe(check_remove_dup))

Footfall hasn't changed when combining cameras
There are 0 duplicates left


Have a look at head, cols etc

In [3]:
footfalldf.head()

,Location,Date,Hour,Count,DateTime,FileName
0,Albion Street South,2020-08-03,0,3,2020-08-03 00:00:00,03.08.20%20-09.08.20.csv
1,Albion Street South,2020-08-03,1,6,2020-08-03 01:00:00,03.08.20%20-09.08.20.csv
2,Albion Street South,2020-08-03,2,1,2020-08-03 02:00:00,03.08.20%20-09.08.20.csv
3,Albion Street South,2020-08-03,3,30,2020-08-03 03:00:00,03.08.20%20-09.08.20.csv
4,Albion Street South,2020-08-03,4,20,2020-08-03 04:00:00,03.08.20%20-09.08.20.csv


In [4]:
footfalldf.columns

Index(['Location', 'Date', 'Hour', 'Count', 'DateTime', 'FileName'], dtype='object')

Check for missing values

In [5]:
footfalldf.isnull().sum()

Location    0
Date        0
Hour        0
Count       0
DateTime    0
FileName    0
dtype: int64

Output to CSV file for analysis elsewhere

In [6]:
footfalldf.to_csv("./data/lcc_footfall_combined.csv")

Created a function that creates a summary of total footfall and the mean of all instances of a time interval
(ie. all weekdays in camera history).

In [21]:
def create_sum_df(data):
    time_dico = {
        "interval": ["hours", "day", "week", "month", "year"],
        "code": ["%H", "%a", "%W", "%b", "%y"],
        "freq": ["H", "D", "W", "MS", "Y"]
    }

    df_dico = {}
    for i, time in enumerate(time_dico['interval'][1:]):
        data_resampled = data.set_index('DateTime').resample(time_dico['freq'][i]).sum()
        if time == 'day':
            data_resampled['weekday'] = data_resampled.index.dayofweek
            data_resampled['weekdayname'] = data_resampled.index.day_name()
            data_resampled = data_resampled.groupby(['weekday', 'weekdayname'])['Count'].agg(
                ['sum', 'mean']).droplevel(level=0)
        elif time == 'week':
            data_resampled['weekno'] = data_resampled.index.isocalendar().week
            data_resampled = data_resampled.groupby(['weekno'])['Count'].agg(['sum', 'mean'])
        elif time == 'month':
            data_resampled['month'] = data_resampled.index.month
            data_resampled['monthname'] = data_resampled.index.month_name()
            data_resampled = data_resampled.groupby(['month','monthname'])['Count'].agg(['sum', 'mean']).droplevel(level = 0)
        elif time == 'year':
            data_resampled['year'] = data_resampled.index.year
            data_resampled = data_resampled.groupby(['year'])['Count'].sum()

        df_dico[time] = data_resampled
    return df_dico

In [23]:
#Initialise a dictionary mapping the intervals, date format codes and resampling frequencies for us in functions and loops.

fig = px.bar(create_sum_df(footfalldf)['day'],
              title="Bar to show days",
             y='mean')

fig.show()

In [8]:
fig = px.bar(create_sum_df(footfalldf)['week'],
              title="Bar to show days",
             y='mean')

fig.show()

In [20]:
print(create_sum_df(footfalldf)['month'])
fig = px.bar(create_sum_df(footfalldf)['month'],
              title="Bar to show months",
             y='mean')

fig.show()



                sum          mean
monthname                        
January    53417993  9.209999e+05
February   50345240  9.681777e+05
March      57225411  9.866450e+05
April      51500557  9.363738e+05
May        58728939  1.087573e+06
June       56254306  1.103026e+06
July       59902809  1.130242e+06
August     61428545  1.137566e+06
September  59972604  1.070939e+06
October    57110220  1.019825e+06
November   59659377  1.046656e+06
December   73124217  1.282881e+06


The bar charts below show mean hourly footfall over each month for 2018-2021.  This has helped to identify gaps in
the data so creating a more complete file.  It's also really interesting to see the immediate impacts lockdown had on
footfall, even in late March when restrictions were only implemented in the very last week of the month.

Would be good to quantify just how much footfall has fallen compared to the same months in previous years,
also to see how much it fell compared to changes between months across the years.

In [45]:
def monthly_mean_hourly(dataf,year):
    dataf = dataf.set_index('DateTime').groupby(
    [pd.Grouper(level="DateTime",
                freq="D")]).aggregate(np.mean).rename(columns={'Count': 'Mean Hourly Footfall'})

    dataf = dataf.loc[dataf.index.year >= year]


    return dataf

fig = px.bar(monthly_mean_hourly(footfalldf,2018),
             x = monthly_mean_hourly(footfalldf,2018).index.month_name(),
             y='Mean Hourly Footfall',facet_row=monthly_mean_hourly(footfalldf,2018).index.year,)

fig.show()


In [43]:
fig = px.line(monthly_mean_hourly(footfalldf,2016),
              title="Comparison of Mean Hourly footfall over 12 months",
              x=monthly_mean_hourly(footfalldf,2016).index.month_name(),
              y='Mean Hourly Footfall',
              color=monthly_mean_hourly(footfalldf,2016).index.year,
              )

fig.show()

In [98]:
import datetime


#print(daily_mean_hourly(footfalldf,2020))

fig = px.line(daily_mean_hourly(footfalldf,2020),
              title="Comparison of Mean Hourly footfall over 12 months",
              y='Mean Hourly Footfall')

fig.add_vline(x=datetime.datetime.strptime('2020-03-20',"%Y-%m-%d").timestamp() * 1000,
              line_color="green", line_dash="dash",
              annotation_text="20th March <br> PM announces <br> hospitality to close <br> at midnight",
              annotation_font_size=10,
              annotation_textangle=0,
              annotation_position="top"),

fig.add_vline(x=datetime.datetime.strptime('2020-03-23',"%Y-%m-%d").timestamp() * 1000,
              line_color="green", line_dash="dash",
              annotation_text="23rd March <br> PM announces <br> first lockdown",
              annotation_font_size=10,
              annotation_textangle=0,
              annotation_position="top"),

fig.add_vline(x=datetime.datetime.strptime('2020-06-15',"%Y-%m-%d").timestamp() * 1000,
              line_color="green", line_dash="dash",
                annotation_text="15th June <br> non-essential <br> shops reopen",
              annotation_font_size=10,
              annotation_position="top")




fig.show()



Playing around with Tableau Public integration to compare viz, pretty sweet.

In [97]:
%%html
<div class='tableauPlaceholder' id='viz1623410630359' style='position: relative'><noscript><a href='#'><img alt='Sheet 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;LI&#47;LIDA2-ambientpopulations&#47;Sheet1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='LIDA2-ambientpopulations&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;LI&#47;LIDA2-ambientpopulations&#47;Sheet1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1623410630359');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

This section will contain an analysis of changes in footfall against introductions or lifting of lockdown restrictions.  Some ideas for how to present this:

    - Add vertical lines at specific points where restrictions were introduced/added
    - Quantify changes in footfall from weeks before to after at various key points.
    - Quantify changes over specific days eg Monday prior compared to Monday after.  See whether this impacts any existing trends, for example if footfall is generally higher on any specific days of the week, do the restrictions change this in any way?
    - Does this keep decreasing or were there fluctuations that might be explained by further changes or external factors?
